In [1]:
library(dplyr)
library(plsRglm)


Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



In [56]:
get_default_hyperparameters <- function(){
    hyperparameters <- list()
    attr(hyperparameters, "n_attempts") <- 10
    attr(hyperparameters, "single_model_K") <- 10
    attr(hyperparameters, "pls_components") <- 2
    attr(hyperparameters, "pls_model") <- "pls-glm-gaussian"
    attr(hyperparameters, "use_wvc") <- TRUE
    attr(hyperparameters, "retest_predictors") <- FALSE
    return(hyperparameters)
}

In [ ]:
train_ensemble <- function(data, response_name="quality", hyperparameters=get_default_hyperparameters()){
    
    response <- data[[response_name]]
    predictors <- data
    predictors[[response_name]] <- NULL
    
    znorm <- function(x)  (x - mean(x))/ sd(x)
    predictors <- apply(predictors, 2, znorm)
    print(head(predictors))
    
}

In [ ]:
#build a single model given a description
build_single_model <- function

In [54]:
#train a single model
#to serve as part of an ensemble
#return set of predictors used for that model
train_single_model <- function(predictors, response, hyperparameters=get_default_hyperparameters()){
    
    K <- attr(hyperparameters, "single_model_K")
    CV_folds   <- get_CV_folds(nrow(predictors), K)
    train_indices <- CV_folds[1]
    test_indices <- CV_folds[2]
    n_attempts <- attr(hyperparameters, "n_attempts")
    n_comp <- attr(hyperparameters, "pls_components")
    modele <- attr(hyperparameters, "pls_model")
    retest <- attr(hyperparameters, "retest_predictors")
    
    use_wvc <- att(hyperparameters, "use_wvc")
    
    if(use_wvc){
        plsRglm_func = plsRglm_wvc
    }
    else{
        plsRglm_func = plsRglm
    }
    
    predictor_set <- list()
    best_predictor_set <- list()
    tested_predictors <- list()
    best_score <- Inf
    
    for(n in 1:n_attempts){
        
        current_predictor_set <- best_predictor_set
        
        #for the first iteration, run the empty model
        #to get performance of intercept
        #after first iteration, add predictors to model
        if(n > 1){
            all_predictors <- colnames(predictors)
            unused_predictors <- list()
            for j in 1:length(unused_predictors){
                p <- all_predictors[j]
                if(!( p %in% current_predictor_set) && (retest || !(p %in% tested_predictors))){
                    unused_predictors <- append(unused_predictors, p)
                }
            }
            p <- sample(unused_predictors, 1)
            current_predictor_set <- append(current_predictor_set, p)
            tested_predictors <- p
        }
        
        
        oof_scores <- vector("list", K)
        
        for(k in 1:K){
            train_X <- predictors[train_indices,current_predictor_set]
            test_X  <- predictors[test_indices,current_predictor_set]
            train_Y <- response[train_indices,]
            test_Y  <- response[test_indices,] 

            modplsglm  <- plsRglm_func(train_Y,train_X,dataPredictY=test_X,nt=n_comp,modele=modele)
            
            oof_scores[i] <- mean(abs(test_Y - modplsglm$ValsPredictY))
        }
        #if the model is an improvement over previous best, update it
        final_score <- mean(oof_scores)
        if(final_score < best_score){
            best_score <- final_score
            best_predictor_set <- current_predictor_set
        }
    }
    #train and evaluate the model on the full dataset?
    #use this to get residuals for use by future models?
    
    return(best_predictor_set)
}

ERROR: Error in parse(text = x, srcfile = src): <text>:34:9: unexpected '}'
33:             oof_scores[i] <- modplsglm$
34:         }
            ^


In [4]:
find_or_download_data <- function (main_url, files){

    for (i in 1:length(files)){
        if(!file.exists(files[i])){
            print(paste("Could not find", files[i], "- attempting to download", sep=" " ))
            download.file(paste(main_url, files[i],sep="") , files[i], "auto", quiet = FALSE)
        }
    }
    for (i in 1:length(files)){
        if(!file.exists(files[i])){
            print(paste("Could not find or download", files[i], "- training will fail", sep=" " ))
            return(0)
        }
    }
    return(1)
}

In [39]:
get_CV_folds <- function(N, K){
    train_indices <- vector("list", K)
    test_indices  <- vector("list", K)
    random_permutation <- sample(N, N, replace=FALSE)
    for (i in 1:K){
        start <- floor(N*(i-1)/K) + 1
        stop  <- floor(N*i/K)
        test_indices[i] <- random_permutation[start:stop]
        train_indices[i] <- random_permutation[-(start:stop)]
    }
    return(c(train_indices, test_indices))
}

In [2]:
#script for evaluating different approaches
dev_script <- function(){
    main_url <- "https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/"
    files <- c("winequality-red.csv", "winequality-white.csv")
    K <- 5
    #number of folds for topmost level of cross-validation
    
    have_data <- find_or_download_data(main_url, files)
    if(!have_data){
        print("Could not find or download data - returning null")
        return(NULL)
    }
    else{
        single_datasets <- vector("list", length(files))
        for (i in 1:length(files)){
            single_datasets[[i]] <- read.csv(files[i], header=TRUE, sep=";")
            single_datasets[[i]]$file.number <- i
        }
        data <- cbind(single_datasets)
        data <- bind_rows(single_datasets, .id = "column_label")
        #data = full dataset (merged across all files)
        
        CV_indices <- get_CV_folds(nrow(data), K)
        train_indices <- CV_indices[1]
        test_indices  <- CV_indices[2]
        
        for (i in 1:K){
            ensemble <- train_ensemble(data[train_indices[i],:], response="quality")
            oof_pred <- predict(ensemble, data[test_indices[i],:], response="quality")
        }
        
        
        n_models <- 10
        
        
        
        model <- train_single_model(data, "quality")
        
        
    }
    return(NULL)
}

In [3]:
sd$quality

ERROR: Error in sd$quality: object of type 'closure' is not subsettable


In [5]:
sd <- train_model()

In [5]:
df <- read.csv("winequality-red.csv", header=TRUE, sep=";")

In [10]:
df2 <- df
df2$quality <- NULL
head(df)

fixed.acidity,volatile.acidity,citric.acid,residual.sugar,chlorides,free.sulfur.dioxide,total.sulfur.dioxide,density,pH,sulphates,alcohol,quality
7.4,0.70,0.00,1.9,0.076,11,34,0.9978,3.51,0.56,9.4,5
7.8,0.88,0.00,2.6,0.098,25,67,0.9968,3.20,0.68,9.8,5
7.8,0.76,0.04,2.3,0.092,15,54,0.9970,3.26,0.65,9.8,5
11.2,0.28,0.56,1.9,0.075,17,60,0.9980,3.16,0.58,9.8,6
7.4,0.70,0.00,1.9,0.076,11,34,0.9978,3.51,0.56,9.4,5
7.4,0.66,0.00,1.8,0.075,13,40,0.9978,3.51,0.56,9.4,5


In [17]:
znorm <- function(x)  (x - mean(x))/ sd(x)
dfnorm <- apply(df, 2, znorm)

In [48]:
df2[["quality"]] <- NULL

In [57]:
nrow(df2)

[1] 1599

In [62]:
colnames(df2)

[1] "fixed.acidity"        "volatile.acidity"     "citric.acid"         
 [4] "residual.sugar"       "chlorides"            "free.sulfur.dioxide" 
 [7] "total.sulfur.dioxide" "density"              "pH"                  
[10] "sulphates"            "alcohol"

In [38]:
N = 33
K = 5
order <- sample(N,N,replace=FALSE)
print(order)
for (i in 1:K){
    start <- floor(N*(i-1)/K) + 1
    stop  <- floor(N*i/K)
    print(order[start:stop])
    print(order[-(start:stop)])
    #print(c(order[0:(start-1)], order[stop+1:length(order)]))
}


 [1] 19 33 10 26 18 24 21 14  4 27  2 15  7 11 22 30  9 32  3 23  1  5 29  6 25
[26] 13 17  8 20 31 16 28 12
[1] 19 33 10 26 18 24
 [1] 21 14  4 27  2 15  7 11 22 30  9 32  3 23  1  5 29  6 25 13 17  8 20 31 16
[26] 28 12
[1] 21 14  4 27  2 15  7
 [1] 19 33 10 26 18 24 11 22 30  9 32  3 23  1  5 29  6 25 13 17  8 20 31 16 28
[26] 12
[1] 11 22 30  9 32  3
 [1] 19 33 10 26 18 24 21 14  4 27  2 15  7 23  1  5 29  6 25 13 17  8 20 31 16
[26] 28 12
[1] 23  1  5 29  6 25 13
 [1] 19 33 10 26 18 24 21 14  4 27  2 15  7 11 22 30  9 32  3 17  8 20 31 16 28
[26] 12
[1] 17  8 20 31 16 28 12
 [1] 19 33 10 26 18 24 21 14  4 27  2 15  7 11 22 30  9 32  3 23  1  5 29  6 25
[26] 13
